부정리뷰 대비 추가사항
+ [ ] 키워드 추출 : ChatGPT
+ [ ] 키워드에 대한 하이라이트 처리 : mark(CSS)
+ [x] 답변 내용에 대한 예시 Append : ChatGPT

In [1]:
###############
# 1. import
#####

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import time 

from bs4 import BeautifulSoup
from selenium import webdriver
from skimage import io

# local only
import os
from os import listdir

import pymysql

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)

import send_email as se
from jinja2 import Template

# 한글정제 KoNLPy
from ckonlpy.tag import Twitter
twitter = Twitter()


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:

###############
# 2.자료 추출
#####

# DB연결
conn = pymysql.connect(host = '10.223.7.4', user ='MAEIL_CS', password = "Maeil01!@", database='MAEIL_CS')

# cursor 설정
cursor = conn.cursor(pymysql.cursors.DictCursor)


#날짜 확인
sql= '''select DATE_FORMAT (CURDATE(), '%a') as week'''
cursor.execute(sql)
gubun = cursor.fetchall()
gubun = pd.DataFrame(gubun)
gubun = gubun['week'][0]


In [3]:
#자료 불러오기 : 전날 긍정리뷰 겟수

if gubun =='Mon':
    sql= '''select distinct MAPPING_KEY as "상품번호"
        , A.WRITER as "등록자" 
        , A.WRITE_DT as "리뷰등록일"
        , A.CONTENTS as "리뷰상세내용"
        , A.REVIEW_NO as "리뷰글번호"
        FROM MAEIL_REVIEW A
        where GRADE in (5)
        and A.COMPANY_CODE in (13358)
        and CHAR_LENGTH(CONTENTS) >= 300
        and STR_TO_DATE(WRITE_DT, '%Y%m%d') between DATE_SUB(CURDATE(), INTERVAL 3 DAY) and (CURDATE())
        '''
else:
    sql= '''select distinct  MAPPING_KEY as "상품번호"
        , A.WRITER as "등록자" 
        , A.WRITE_DT as "리뷰등록일"
        , A.CONTENTS as "리뷰상세내용"
        , A.REVIEW_NO as "리뷰글번호"
        FROM MAEIL_REVIEW A
        where GRADE in (5)
        and A.COMPANY_CODE in (13358)
        and CHAR_LENGTH(CONTENTS) >= 300
        and STR_TO_DATE(WRITE_DT, '%Y%m%d') between DATE_SUB(CURDATE(), INTERVAL 1 DAY) and (CURDATE())
        '''

In [4]:
cursor.execute(sql)
result7 = cursor.fetchall()
result7 = pd.DataFrame(result7)

result7_cnt = len(result7)


In [5]:
result7_cnt

0

In [7]:
if result7_cnt < 2:
    if gubun =='Mon':
        sql= '''select distinct MAPPING_KEY as "상품번호"
            , A.WRITER as "등록자" 
            , A.WRITE_DT as "리뷰등록일"
            , A.CONTENTS as "리뷰상세내용"
            , A.REVIEW_NO as "리뷰글번호"
            FROM MAEIL_REVIEW A
            where GRADE in (5)
            and A.COMPANY_CODE in (13358)
            and CHAR_LENGTH(CONTENTS) >= 100
            and STR_TO_DATE(WRITE_DT, '%Y%m%d') between DATE_SUB(CURDATE(), INTERVAL 3 DAY) and (CURDATE())
            order by CHAR_LENGTH(CONTENTS) DESC 
            limit 10
            '''
    else:
        sql= '''select distinct  MAPPING_KEY as "상품번호"
            , A.WRITER as "등록자" 
            , A.WRITE_DT as "리뷰등록일"
            , A.CONTENTS as "리뷰상세내용"
            , A.REVIEW_NO as "리뷰글번호"
            FROM MAEIL_REVIEW A
            where GRADE in (5)
            and A.COMPANY_CODE in (13358)
            and CHAR_LENGTH(CONTENTS) >= 100
            and STR_TO_DATE(WRITE_DT, '%Y%m%d') between DATE_SUB(CURDATE(), INTERVAL 1 DAY) and (CURDATE())
            order by CHAR_LENGTH(CONTENTS) DESC 
            limit 10
            '''
    cursor.execute(sql)
    result7 = cursor.fetchall()
    result7 = pd.DataFrame(result7)
    result7 = result7.head(2)
    result7_cnt = len(result7)

In [9]:
result7

,상품번호,등록자,리뷰등록일,리뷰상세내용,리뷰글번호
0,2540929238,undt****,20230320,매일유업은 다른 브랜드들에 비해 우유 대체품을 많이 내줘서 자주 사먹게 됩니다. 매...,4213990766
1,5794224924,co2b****,20230320,당뇨 전단계이신 어머니를 위해 주문하면서 겸사겸사 저도 한번 먹어봤슴다.\n숟가락이...,4213963299


In [10]:
result7['리뷰상세내용'] = result7['리뷰상세내용'].str.replace('\n', ' ')

t


In [12]:
if result7_cnt == 0:
    result7 = pd.DataFrame({ '수집체널': ['(해당사항 없음)'], '내용': ['(해당사항 없음)'] })


In [69]:
import openai
from openai.embeddings_utils import cosine_similarity, get_embedding

# Set the API key
openai.api_key = '************************************************' 
model_engine = 'text-davinci-003'

def chatGTP_example(text):
    prompt = '다음 내용에 대해, 최대한 공손한 말투로 적절한 답변 예시를 작성해 줘. ' + text
    completion = openai.Completion.create(engine=model_engine, prompt=prompt, max_tokens=1024, n=1, stop=None, temperature=0.8)
    message = completion.choices[0].text
    return message

In [72]:
model_engine = 'text-davinci-003'
response_example = []
# for i in range(len(result7['리뷰상세내용'])):
#     try:
#         resp_text = chatGTP_example(result7['리뷰상세내용'][i])
#     except:
#         resp_text = '(timeout)'
#     response_example.append(resp_text)
for i in range(len(result7['리뷰상세내용'])):
    resp_text = chatGTP_example(result7['리뷰상세내용'][i])
    response_example.append(resp_text)

result7['답변예시(text-davinci-003)'] = response_example

In [73]:
result7

,상품번호,등록자,리뷰등록일,리뷰상세내용,리뷰글번호,답변예시(text-davinci-003)
0,2540929238,undt****,20230320,매일유업은 다른 브랜드들에 비해 우유 대체품을 많이 내줘서 자주 사먹게 됩니다. 매...,4213990766,\n\n감사합니다. 매일유업은 최상의 제품을 제공하기 위해 노력하고 있으며 이를 바...
1,5794224924,co2b****,20230320,당뇨 전단계이신 어머니를 위해 주문하면서 겸사겸사 저도 한번 먹어봤슴다. 숟가락이 ...,4213963299,\n\n감사합니다. 당뇨 전단계이신 어머니를 위해 주문해 주셔서 감사합니다. 맛이 ...


In [75]:
model_engine = 'gpt-3.5-turbo-0301'
response_example = []
for i in range(len(result7['리뷰상세내용'])):
    resp_text = chatGTP_example(result7['리뷰상세내용'][i])
    response_example.append(resp_text)
    
result7['답변예시(gpt-4-0314)'] = response_example

InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [147]:
result7

,상품번호,등록자,리뷰등록일,리뷰상세내용,답변예시(text-davinci-003),답변예시(text-davinci-002),답변예시(text-curie-001)
0,8091416321,youn*******,20230308,건강검진때마다 콜레스테롤이 높게 나오고 해서 주변에서 이걸 추천받아서 구매하게되었네...,\n\n감사합니다. 콜레스테롤 농도를 낮추기 위해 구매하신 건 좋은 생각이네요. 마...,\n\n콜레스테롤이 높게 나오고 해서 주변에서 이걸 추천받아서 구매하게되었네요. ...,\n\nI would like to submit a response to the a...
1,7660900915,anzz****,20230308,도대체 무슨 맛일까 궁금해서 주문했는데 저는 초코맛보다 이쪽이 더 마음에 들어요. ...,\n\n복숭아맛과 단백질음료 특유의 맛이 잘 어우러져 맛있게 느껴졌어요. 당에 대해...,\n\nThe taste is sort of like a cross between ...,저는 이런 단맛을 대체하게 되는 방법이 아니었어요.\n\nI usually ord...
2,7190760869,chla********,20230308,평소에 우유를 많이 먹는데 우유보다는 아몬드브리즈가 더 맛있고 몸에도 좋을 것 같아...,\n\n감사합니다. 아몬드브리즈는 배가 안 아프고 속도 차서 좋기 때문에 좋은 선택...,\n\nIt's good to hear that you're enjoying the...,\n\nI try to eat a lot of fruit because it is ...
3,6223774804,muji****,20230308,주기적으로 구매하는 상품입니다. 가격 맛 모두 만족스러워요. 유통기한은 23년 7월...,"\n\n감사합니다. 제품의 맛과 가격, 유통기한 등 모두 만족하고 계신거 같습니다....",\n\nThank you for your kind words! We're so gl...,아무도 아주 좋은 파우치에 찾아왔습니다. 아무도 많은 사람들이 이번에 아무도 없습...
4,6208609088,lila********,20230308,친구네 집에서 마셔보고 깜짝 놀라서 바로 주문했어요. 이제 카페 갈 일 없겠어요! ...,"\n\n네, 엄마님 말씀 따르겠습니다. 더 많은 사람들에게 즐거운 커피 타임을 선물...",친구들 다 만족하시고 있어요. 아이스로 마시면 좋아요.,그리고 집에서 만들어 드리는 아이스를 위해 마신다면 제가 아무것도 알 수 없어요....
5,6178251164,hyeo******,20230308,빠른배송 감사드려요!! 집에오는길에 맨날 들려서 동물들어간 킨더조이 사달라고 졸라...,\n\n감사합니다. 그럼 이번에는 빠른 배송 덕분에 만족하게 받아 보셨으면 좋겠습니...,이거 다 같은곳에서 사세요!\n\n저도 마찬가지입니다. 집에서 가까운 곳에서 편하...,.\n\nI am very sorry that I didn't get the ani...
6,6095910758,royh****,20230308,세번째 구매입니다. 그릭요거트를 매일 먹기로 결심하고 이 제품 저 제품 시식해보다 ...,\n\n네 노력이 보여지는군요. 직접 시식해 보시고 결정해 주신 건 덕분에 이 제품...,세번째 배송입니다. 구매하시고 한 달 간 먹으시면 느끼시는 차이가 무척 많이 있으...,제품 이미지를 참고해 드립니다.\n\nI am writing to express ...
7,5828829127,kaya****,20230308,"건강 위해 라떼용으로 사기 시작했어요.\n처음엔 적응 못 했는데, \n우유랑 섞어 ...","\n\n정말 다행이네요. 우유랑 섞어도 맛있게 먹을 수 있고, 완전 맛있는 라떼를 ...",\n\nQ. 요즘 라떼용으로 사기 시작했어요. 처음엔 적응이 안 되었어요. 우유랑 ...,\n\nI originally started using it to get healt...
8,5828829127,rmaq****,20230308,편의점에서 매일 사먹다가 이렇게 먹을바엔 한박스 시키자! 하고 시켰어요 ㅎㅎ 밤10...,\n\n저도 귀여운 젖소 박스에서 매일 맛있는 음식을 먹어보고 싶어요! 총알배송이라...,\n\n1. Have you tried the new cereal at the co...,
9,5828824627,mjki*****,20230308,배송도 빨랐고 상자에 캐릭터도 너무 귀여워요( &curren;̴̶̷̤́ ‧̫̮ &c...,"\n\n네, 매우 반갑습니다. 저희 제품에 대한 만족감을 느끼고 고객님의 높은 기술...",!\n\n고객님께서 주문해주신 제품이 마시시간에 왔다는 것은 저희 상점에 고객님께서...,~\n\n\n\nPlease give me your honest feedback a...


In [ ]:


# if result7_cnt != 0:
#     response_example = []
#     for i in range(len(result7['리뷰상세내용'])):
#         try:
#             resp_text = chatGTP_example(result7['리뷰상세내용'][i])
#         except:
#             resp_text = '(timeout)'
#         response_example.append(resp_text)
        
#     result7['답변예시(text-davinci-003)'] = response_example
#     result7['리뷰상세내용'] = result7['리뷰상세내용'].str.replace('\n', ' ')
# else:
#     result7 = pd.DataFrame({ '수집체널': ['(해당사항 없음)'], '내용': ['(해당사항 없음)'] })

In [139]:
to_users = 'cappygirl@maeil.com; gmlrud852@maeil.com;'
to_cc = "khpark@maeil.com"

In [140]:
###############
# 3.이메일 발송
#####

# 이메일 양식(내용) frame 구성하기
message_tmpl="""
<html>
    <!DOCTYPE html>
    <html>
    <head lang ="ko">
      <meta charset ="UTF-8">
      <title>{{title1}}</title>
    </head>
    <BODY style="FONT-SIZE: 10pt; font-family: 맑은고딕, Malgun Gothic, dotum, gulim, sans-serif">
    안녕하세요,<br>
    <br>{{send_date}}시 기준, <mark>네이버</mark> 직영채널의 우호적 리뷰 내용 공유드립니다.<br>
    (우호적 리뷰 기준 : 평점 5점 & 내용이 300자 이상)<br>
    <br>
    <b>1. 우호적 리뷰 : {{count1}} 개</b><br>
    <br>
    <b>2. 우호적 리뷰 상세</b><br>
    {{ df_html }}<br>
    <br>
    <SPAN style="COLOR: #808080">{{add_text1}}</SPAN><br>
    <SPAN style="COLOR: #808080">{{add_text2}}</SPAN><br>
    <br>
    감사합니다.<br>
    <br>
    </BODY>
</html>"""

# 이메일 양식(내용) 변동 내용 정의
render_data = {
    'send_date' : (datetime.today()).strftime('%Y-%m-%d, %H'),
    'df_html' : result7.to_html().replace('무색', '<mark>무색</mark>'),
    'count1' : result7_cnt,
    'add_text1' : "*  본 메일은 발신전용입니다.",
    'add_text2' : "** (내용/기술문의 : 고객정보전략담당, 박경하 )"
}


t= Template(message_tmpl)
html_report = t.render(render_data)

In [141]:
# 메일제목
now = datetime.today()    # 현재
date = now.strftime('%Y%m%d')

subject = "(TEST)[자동발송] 네이버 직영채널의 우호적 리뷰 관리 List"

# 본문
html = html_report

# 첨부파일 추가
try:
    attach = directory_name + file_name
except:
    attach = None
graph=None

#메일 발송
import win32com.client

outlook = win32com.client.Dispatch("Outlook.Application")
send_mail = outlook.CreateItem(0)

send_mail.To = to_users #메일 수신인
send_mail.CC = to_cc #메일 수신인

send_mail.Subject = subject #메일 제목
send_mail.HTMLBody =html_report
# try:
#     send_mail.Attachments.Add(Source=str(attach))
# except:
#     pass

send_mail.Display(True)

In [117]:
# send_mail.Display(True)

In [83]:
src_soup = BeautifulSoup(html_report, 'html.parser')
dst_soup = BeautifulSoup('', 'html.parser')

In [84]:
text_highlight = ['빠른', '배송', '넉넉']

In [85]:
def mark_if_needed(text):
    # can be improved using regex
    for word in text_highlight:
        if word in text:
            return '<mark>' + text + '</mark>'
    return text


In [86]:
p_elements = src_soup.find_all('p')
for p in p_elements:
    a_elements = p.find_all('a')
    p.string = mark_if_needed(p.text)
    dst_soup.append(p)
    for a in a_elements:
        a.string = mark_if_needed(a.text)
        p.append(a)

In [93]:
def mark_if_needed(text):
    doc = nlp(text)
    for sent in doc.sents:
        check = re.search(text_highlight[0], sent.text)
        if check is None:
            yield (0, sent.text)
        else:
            yield (1, sent.text)

p_elements = src_soup.find_all('p')
for p in p_elements:
    s = BeautifulSoup()
    pp = BeautifulSoup()
    par = pp.new_tag('p')

    for sent in mark_if_needed(p.text):
        if sent[0] is 1:
            m = s.new_tag('mark') 
            m.append(sent[1])
            par.append(m)

        else:
            par.append(sent[1])

    dst_soup.append(par)

print(dst_soup.prettify())
html = dst_soup.prettify("utf-8")
with open("output.html", "wb") as file:
    file.write(html)